# Ejercicio de multiagentes con Handsoff 
En este ejercicio vamos a utilizar la herramienta de [Google de ADK](https://google.github.io/adk-docs/) para la creación de tareaas con multi-agentes y un handsoff para la coordinación de las herramientas que puedan utilizar. 
Para esto necesitamos python 3.9 o superior.

Para la instalacion lo realizamos con:
```shell
pip install google-adk
```

## ¿Por qué usar ADK?
ADK es otro framework como son los SDK pero de Google, este tiene ciertas facilidades a la hora de crear nuestro multi-agente y son:

### Es multi-agente por diseño
* Crea sistemas de agentes especializados que colaboran
* Orquestación paralela, secuencial o jerárquica
* Modularidad y escalabilidad integradas

### Mayor flexibilidad en los modelos
* Agnóstico en el uso de tecnología como Llama, CLaude, GPT, Gemini...
* Integraicón con LiteLLM para máximca compatibilidad
* Cambiode modelos isn modificar la arquitectura

### Ecosistema de herramientas
* Herramientas precosntruidas (búsquedas, códigos, etc...)
* Funciones personalizadas fáciles de crear
* Integración con LangChain y LlamaIndex

### Orquestación Flexible
* Agente de flujo de trabajo (workflow agents)
* Enrutamiento dinámico con LLM
* Control preciso del comportamiento

### Experiencia de desarrollol
* CLI y UI web integradas
* Depuaraicón visual
* Evaluación incoporada
* Despliegue simplificado

In [35]:
import sys
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from getpass import getpass

# Workaround para el problema de google-adk
import requests
import gradio as gr
# Asegurar que requests.exceptions esté disponible antes de importar google-adk
if not hasattr(requests, 'exceptions'):
    from requests import exceptions
    requests.exceptions = exceptions

try:
    from google.adk.sessions import InMemorySessionService
    from google.adk.agents import LlmAgent
    from google.adk.tools import google_search
    from google.genai import types
    from google.adk.runners import Runner
    from google.adk.code_executors import BuiltInCodeExecutor
    print("✅ google.adk importado correctamente con workaround")
except Exception as e:
    print(f"❌ Error incluso con workaround: {e}")


✅ google.adk importado correctamente con workaround


In [36]:
load_dotenv(override=True)
# Verificar LiteLLM
try:
    import litellm
    print("✅ LiteLLM instalado")
except ImportError:
    print("❌ LiteLLM no instalado - ejecuta: pip install litellm")

# Verificar OpenAI
try:
    import openai
    print("✅ OpenAI instalado")
except ImportError:
    print("❌ OpenAI no instalado - ejecuta: pip install openai")

# Probar modelos
modelos_openai = [
    "gemini-1.5-flash"
]

for modelo in modelos_openai:
    try:
        test_agent = LlmAgent(
            name="test",
            model=modelo,
            instruction="Test"
        )
        print(f"✅ Modelo {modelo} es válido")
        break
    except Exception as e:
        print(f"❌ Modelo {modelo} falló: {e}")

✅ LiteLLM instalado
✅ OpenAI instalado
✅ Modelo gemini-1.5-flash es válido


In [37]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key and claude_api_key and gemini_api_key:
    print("❌ No se ha encontrado la clave de API. Por favor, establece la variable de entorno OPENAI_API_KEY.")
    sys.exit(1)
else:
    print(f"✅ Clave de API cargada correctamente. Longitud: {len(openai_api_key)} caracteres")
    print(f"✅ Clave de API cargada correctamente. Longitud: {len(claude_api_key)} caracteres")
    print(f"✅ Clave de API cargada correctamente. Longitud: {len(gemini_api_key)} caracteres")

✅ Clave de API cargada correctamente. Longitud: 164 caracteres
✅ Clave de API cargada correctamente. Longitud: 108 caracteres
✅ Clave de API cargada correctamente. Longitud: 39 caracteres


In [38]:
AGENT_NAME = "calculator_agent"
APP_NAME = "calculator"
USER_ID = "user_001"
SESSION_ID = "session_code_exec_aync"
GEMINI_MODEL = "gemini-1.5-flash"
# CLOUDE_MODEL = "claude-sonnet-4-20250514"
OPENAI_MODEL_FOR_ADK = "openai/gpt-4o-mini"  # Modelo de OpenAI
OPENAI_MODEL_FOR_GRADIO = "gpt-4o-mini"  # Modelo de OpenAI para Gradio

code_agent = LlmAgent(
    name=AGENT_NAME,
    model=GEMINI_MODEL,
    code_executor=BuiltInCodeExecutor(), # Esto habilita la jecución de código
    instruction = """Eres un agente calculadora, es decir que cuando se te proporcione una expresión matemática o se te hable de calaculos, escribirás y ejecutarás un código para caalcula el resultado. Devolverás únicamente el resultado numérico final como texto plano, sin formato markdown, ni bloques de código. En el caso que se te diga calcular el precio de algo si darás un resultado más detallado con la moneda en Euros (EUR)""",
    description="Ejecuta códigos en python para realizar cálculos matemáticos y devuelve el resultado numérico final.",
)

# Corriendo Sessión
session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    agent=code_agent,
    app_name=APP_NAME,
    session_service=session_service,
)


In [39]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Envía una consulta al agente e imprime la respuesta final."""
    print(f"\n>>> Consulta del usuario: {query}")

    # Prepara el mensaje del usuario en el formato de ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final." # Valor por defecto

    # Concepto clave: run_async ejecuta la lógica del agente y genera eventos.
    # Iteramos a través de los eventos para encontrar la respuesta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Puedes descomentar la línea de abajo para ver *todos* los eventos durante la ejecución
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Contenido: {event.content}")

        # Concepto clave: is_final_response() marca el mensaje que concluye el turno.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Se asume que la respuesta de texto está en la primera parte
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Maneja posibles errores/escalamientos
                final_response_text = f"El agente escaló: {event.error_message or 'Sin mensaje específico.'}"
            # Agrega más validaciones aquí si es necesario (por ejemplo, códigos de error específicos)
            break # Deja de procesar eventos una vez encontrada la respuesta final

    print(f"<<< Respuesta del agente: {final_response_text}")

In [40]:
def buscar_producto_por_nombre(nombre_producto: str) -> dict:
    """Busca un producto por su nombre y devuelve un diccionario con los detalles.
    Se usa esta herramienta si se busca información específica de un producto.

    Args:
        nombre_producto (str): Nombre del producto a buscar. (No sensible a mayúsculas o minúsculas)

    Returns:
        dict: Los siguientes campos dispoibles son:
        - 'status' (str): "success" si se encontró el producto, "error" si no.  
        - 'product' (dict, optional): Detalles del producto si se encontró, o un mensaje de error.
        - 'error_message' (str, optional): Mensaje de error si no se encontró el producto.
    """

    print(f"Buscando producto: {nombre_producto}")

    # Simulación de base de datos de productos
    productos_db = {
    "mesa blanca de comedor": {
    "id": "MBC001",
    "nombre": "Mesa Blanca Comedor NORBERG",
    "precio": 129,
    "stock": 15,
    "características": ["120x80 cm", "Acabado blanco mate", "Tablero MDF", "Estilo escandinavo"]
        },
    "patas para mesa blanca comedor (juego de 4)": {
    "id": "PMC001",
    "nombre": "Patas de Mesa NORBERG (4 unidades)",
    "precio": 35,
    "stock": 50,
    "características": ["Acero blanco pintado", "Altura 72 cm", "Incluye tornillería"]
        },
    "silla de comedor gris": {
    "id": "SCG001",
    "nombre": "Silla de Comedor KLIPPAN Gris",
    "precio": 49,
    "stock": 30,
    "características": ["Tapizado gris oscuro", "Patas de madera de haya", "Diseño ergonómico"]
        },
    "cojín para silla gris": {
    "id": "CSG001",
    "nombre": "Cojín para Silla KLIPPAN",
    "precio": 12,
    "stock": 100,
    "características": ["Espuma viscoelástica", "Funda lavable", "Color gris a juego"]
        },
    "estantería blanca modular": {
    "id": "EMB001",
    "nombre": "Estantería Modular BILLY Blanca",
    "precio": 79,
    "stock": 20,
    "características": ["180x80x28 cm", "5 baldas", "Modular"]
        },
    "balda adicional para estantería blanca": {
    "id": "BAB001",
    "nombre": "Balda Adicional BILLY Blanca",
    "precio": 10,
    "stock": 75,
    "características": ["80x28 cm", "Aglomerado", "Compatible con BILLY"]
        },
    "sofá 2 plazas azul": {
    "id": "S2A001",
    "nombre": "Sofá 2 Plazas EKTORP Azul",
    "precio": 299,
    "stock": 8,
    "características": ["Tela azul lavable", "Relleno de espuma", "Estructura de madera maciza"]
        },
    "funda de recambio para sofá azul": {
    "id": "FSA001",
    "nombre": "Funda Recambio EKTORP Azul",
    "precio": 69,
    "stock": 25,
    "características": ["100% algodón", "Lavable a máquina", "Color azul marino"]
        }
    }

    producto = productos_db.get(nombre_producto.lower())

    if producto:
        return {
            "status": "success",
            "product": producto,
        }
    # Búsqueda aproximada si no hay coincidencia exacta
    for key, producto in productos_db.items():
        if any(word in key for word in nombre_producto.lower().split()):
            return {
                "status": "success",
                "product": producto,
            }
        
        # Si no se encuentra el producto
    return {
        "status": "error",
        "error_message": f"No se encontró el producto '{nombre_producto}'."
    }

    

In [41]:
carrito_compras: list[dict] = []

def agregar_al_carrito(nombre_producto: str, cantidad: int, precio: float) -> dict:
    """Agrega un producto al carrito de compras.

    Args:
        nombre_producto (str): Nombre del producto a agregar.
        cantidad (int): Cantidad del producto a agregar.

    Returns:
        dict: Un diccionario con el estado de la operación y detalles del producto agregado.
    """
    global carrito_compras
    
    # Si no se proporciona precio, buscarlo en la base de datos
    if precio is None:
        resultado_busqueda = buscar_producto_por_nombre(nombre_producto)
        if resultado_busqueda['status'] == 'success':
            precio = resultado_busqueda['product']['precio']
        else:
            return {
                "status": "error",
                "message": f"No se pudo encontrar el producto '{nombre_producto}' para obtener su precio."
            }

    carrito_compras.append({
        "id": None,
        "producto": nombre_producto.lower(),
        "cantidad": cantidad,
        "precio": precio,
    })

    total_items = sum(item['cantidad'] for item in carrito_compras)
    
    return {
        "status": "success",
        "message": f"Producto '{nombre_producto}'. Total de artículos: {total_items}.",
        "carrito": carrito_compras
    }

def ver_carrito() -> dict:
    """Muestra el contenido del carrito de compras.

    Returns:
        dict: Un diccionario con el estado del carrito y sus productos.
    """
    global carrito_compras

    if not carrito_compras:
        return {
            "status": "empty",
            "items": [],
            "total_items": 0,
            "precio": 0,
            "message": "El carrito está vacío."
        }
    
    total_items = sum(item['cantidad'] for item in carrito_compras)

    return {
        "status": "success",
        "items": carrito_compras,
        "total_items": total_items,
        "total_price": sum(item['cantidad'] * buscar_producto_por_nombre(item['producto'])['product']['precio'] for item in carrito_compras if buscar_producto_por_nombre(item['producto'])['status'] == 'success'),
        "items_price": carrito_compras,
        "message": f"Hay {len(carrito_compras)} productos en el carrito, total de artículos: {total_items}."
    }

## Creación de las herramientas de las funciones que ya tenemos

In [42]:

call_agent_async_json = {
    "name": "call_agent_async",
    "description": "Llama al agente para procesar una consulta y devuelve la respuesta final.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Consulta del usuario para el agente."
            },
            "runner": {
                "type": "string",
                "description": "Instancia del Runner que ejecuta el agente."
            },
            "user_id": {
                "type": "string",
                "description": "ID del usuario que realiza la consulta."
            },
            "session_id": {
                "type": "string",
                "description": "ID de la sesión actual."
            }
        },
        "required": ["query", "runner", "user_id", "session_id"]
    }
}

In [43]:
buscar_producto_por_nombre_json = {
    "name": "buscar_producto_por_nombre",
    "description": "Busca un producto por su nombre y devuelve un diccionario con los detalles.",
    "parameters": {
        "type": "object",
        "properties": {
            "nombre_producto": {
                "type": "string",
                "description": "Nombre del producto a buscar. (No sensible a mayúsculas o minúsculas)"
            }
        },
        "required": ["nombre_producto"]
    }
}


In [44]:
agregar_al_carrito_json = {
    "name": "agregar_al_carrito",
    "description": "Agrega un producto al carrito de compras.",
    "parameters": {
        "type": "object",
        "properties": {
            "nombre_producto": {
                "type": "string",
                "description": "Nombre del producto a agregar al carrito."
            },
            "cantidad": {
                "type": "integer",
                "description": "Cantidad del producto a agregar al carrito."
            },
            "precio": {
                "type": ["number", "null"],
                "description": "Precio del producto. Si es null, se buscará en la base de datos."
            }
        },
        "required": ["nombre_producto", "cantidad"]
    }  
}

In [45]:
ver_carrito_json = {
    "name": "ver_carrito",
    "description": "Muestra el contenido del carrito de compras.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}

In [46]:
tools = [
    {
        "type": "function",
        "function": call_agent_async_json
    },
    {
        "type": "function",
        "function": buscar_producto_por_nombre_json
    },
    {
        "type": "function",
        "function": agregar_al_carrito_json
    },
    {
        "type": "function",
        "function": ver_carrito_json
    }
]

In [47]:
tools

[{'type': 'function',
  'function': {'name': 'call_agent_async',
   'description': 'Llama al agente para procesar una consulta y devuelve la respuesta final.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'Consulta del usuario para el agente.'},
     'runner': {'type': 'string',
      'description': 'Instancia del Runner que ejecuta el agente.'},
     'user_id': {'type': 'string',
      'description': 'ID del usuario que realiza la consulta.'},
     'session_id': {'type': 'string',
      'description': 'ID de la sesión actual.'}},
    'required': ['query', 'runner', 'user_id', 'session_id']}}},
 {'type': 'function',
  'function': {'name': 'buscar_producto_por_nombre',
   'description': 'Busca un producto por su nombre y devuelve un diccionario con los detalles.',
   'parameters': {'type': 'object',
    'properties': {'nombre_producto': {'type': 'string',
      'description': 'Nombre del producto a buscar. (No sensible a mayúscu

In [66]:
def handle_tool_call(tool_calls):
    """Maneja la llamada a una herramienta y devuelve el resultado."""
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herramienta llamada: {tool_name} con argumentos: {arguments}", flush=True)

        tool = globals().get(tool_name)
        if tool:
            result= tool(**arguments)
            results.append({
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id
            })
        else:
            results.append({
                "role": "tool",
                "content": json.dumps({"error": f"Herramienta {tool_name} no encontrada"}),
                "tool_call_id": tool_call.id
            })
    return results

## Creamos un agente comercial para nuestra ecommerce


In [62]:
# Agente especializado en e-commerce
agente_ecommerce = LlmAgent(
    name="AgenteEcommerce",
    model=GEMINI_MODEL,  # Usar el modelo con formato correcto
    description="Asistente de compras online",
    tools=[
        buscar_producto_por_nombre,
        agregar_al_carrito,
        ver_carrito
    ],
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2,
        max_output_tokens=400
    ),
    instruction = (
        "Eres un asistente de compras amigable y servicial. "
        "Tu misión es ayudar al usuario a encontrar productos, agregarlos al carrito de forma sencilla "
        "y guiarlo durante el proceso de compra. "
        "Debes comprender peticiones incluso si el nombre del producto no coincide exactamente, "
        "haciendo coincidir términos aproximados con los nombres reales en la base de datos. "
        "Cuando un producto mencionado tenga piezas asociadas o vendidas por separado (como patas o fundas), "
        "sugiere ambas opciones: la compra por separado o como conjunto completo. "
        "Siempre que sea posible, ofrece productos relacionados o complementarios que puedan interesar al usuario. "
        "Ten en cuenta la disponibilidad en stock, e informa si algún producto no está disponible. "
        "Mantén un tono proactivo, claro y amable durante toda la conversación."
    )
)

print("✅ Agente de e-commerce creado")

✅ Agente de e-commerce creado


In [63]:
# Concepto clave: SessionService almacena el historial y estado de la conversación.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de la interacción
APP_NAME = "agente_ecommerce"
USER_ID = "user_4"
SESSION_ID = "004" # Usando un ID fijo por simplicidad

# Crear la sesión específica donde ocurrirá la conversación
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: Este es el componente principal que gestiona la interacción con el agente.
runner = Runner(agent=agente_ecommerce,
                app_name=APP_NAME,
                session_service=session_service)

In [67]:
def chat(message, history):
    messages = [
        {
            "role": "system",
            "content": """Eres un asistente de compras amigable y servicial. "
        "Tu misión es ayudar al usuario a encontrar productos, agregarlos al carrito de forma sencilla "
        "y guiarlo durante el proceso de compra. "
        "Debes comprender peticiones incluso si el nombre del producto no coincide exactamente, "
        "haciendo coincidir términos aproximados con los nombres reales en la base de datos. "
        "Cuando un producto mencionado tenga piezas asociadas o vendidas por separado (como patas o fundas), "
        "sugiere ambas opciones: la compra por separado o como conjunto completo. "
        "Siempre que sea posible, ofrece productos relacionados o complementarios que puedan interesar al usuario. "
        "Ten en cuenta la disponibilidad en stock, e informa si algún producto no está disponible. "
        "Mantén un tono proactivo, claro y amable durante toda la conversación."""
        }
        ] + history + [
        {            
            "role": "user",
            "content": message
        },
    ]

    done = False
    while not done:

        # Vamos a llamar al LLM pasamos el json de la herramienta y la detecta

        response = openai.chat.completions.create(
            model = OPENAI_MODEL_FOR_GRADIO,
            messages = messages,
            tools = tools,
        )

        finish_reason = response.choices[0].finish_reason

        # Si la respuesta es una llamada a una herramienta, la manejamos

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_call(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Herramienta llamada: buscar_producto_por_nombre con argumentos: {'nombre_producto': 'mesa blanca de comedor'}
Buscando producto: mesa blanca de comedor
Herramienta llamada: agregar_al_carrito con argumentos: {'nombre_producto': 'Mesa Blanca Comedor NORBERG', 'cantidad': 2, 'precio': 129}
Herramienta llamada: buscar_producto_por_nombre con argumentos: {'nombre_producto': 'patas para mesa blanca'}
Buscando producto: patas para mesa blanca
Herramienta llamada: agregar_al_carrito con argumentos: {'nombre_producto': 'Juegos de patas para mesa blanca', 'cantidad': 3, 'precio': None}
Buscando producto: Juegos de patas para mesa blanca
Herramienta llamada: buscar_producto_por_nombre con argumentos: {'nombre_producto': 'silla gris'}
Buscando producto: silla gris
Herramienta llamada: buscar_producto_por_nombre con argumentos: {'nombre_producto': 'cojines'}
Buscando producto: cojines
Herramienta llamada: agregar_al_carrito con argumentos: {'nombre_producto': 'Silla de Comedor KLIPPAN Gris', 'cant